In [3]:
import json
import glob
import cv2
import pytesseract
from PIL import Image
import pybboxes as pbx
import re
import numpy as np
import imutils 
import easyocr

def recognize_plate(img, coords):
    

    # from paddleocr import PaddleOCR
    # ocr = PaddleOCR(lang='en',rec_algorithm='CRNN')
    # result = ocr.ocr(img, cls=False, det=False)
    # return result 

    # pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
    # xmin, ymin, xmax, ymax = coords
    # try:
    #     box = img[int(ymin):int(ymax+3), int(xmin):int(xmax)+5]
    #     reader = easyocr.Reader(['en'])
    #     result = reader.readtext(box)
    #     return result[0][1]
    # except:
    #     return ""
# 
    # separate coordinates from box
    xmin, ymin, xmax, ymax = coords
    # print(img)
    print(coords)
    # get the subimage that makes up the bounded region and take an additional 5 pixels on each side
    try:
        box = img[int(ymin):int(ymax), int(xmin)-5:int(xmax)+5]
        # print(box)
        
        # grayscale region within bounding box
        gray = cv2.cvtColor(box, cv2.COLOR_RGB2GRAY)
        # resize image to three times as large as original for better readability
        gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
        # perform gaussian blur to smoothen image
        blur = cv2.GaussianBlur(gray, (5,5), 0)
        #cv2.imshow("Gray", gray)
        #cv2.waitKey(0)
        # threshold the image using Otsus method to preprocess for tesseract
        ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
        #cv2.imshow("Otsu Threshold", thresh)
        #cv2.waitKey(0)
        # create rectangular kernel for dilation
        rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
        # apply dilation to make regions more clear
        dilation = cv2.dilate(thresh, rect_kern, iterations = 1)
        #cv2.imshow("Dilation", dilation)
        #cv2.waitKey(0)
        # find contours of regions of interest within license plate
        try:
            contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        except:
            ret_img, contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # sort contours left-to-right
        sorted_contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])
        # create copy of gray image
        im2 = gray.copy()
        # create blank string to hold license plate number
        plate_num = ""
        # loop through contours and find individual letters and numbers in license plate
        for cnt in sorted_contours:
            try:
                x,y,w,h = cv2.boundingRect(cnt)
                height, width = im2.shape
                # if height of box is not tall enough relative to total height then skip
                if height / float(h) > 6: continue

                ratio = h / float(w)
                # if height to width ratio is less than 1.5 skip
                if ratio < 1.5: continue

                # if width is not wide enough relative to total width then skip
                if width / float(w) > 15: continue

                area = h * w
                # if area is less than 100 pixels skip
                if area < 100: continue

                # draw the rectangle
                rect = cv2.rectangle(im2, (x,y), (x+w, y+h), (0,255,0),2)
                # grab character region of image
                roi = thresh[y-5:y+h+5, x-5:x+w+5]
                # perfrom bitwise not to flip image to black text on white background
                roi = cv2.bitwise_not(roi)
                # perform another blur on character region
                roi = cv2.medianBlur(roi, 5)
                text = pytesseract.image_to_string(roi, config='-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ --psm 8 --oem 3')
                # clean tesseract text by removing any unwanted blank spaces
                clean_text = re.sub('[\W_]+', '', text)
                plate_num += clean_text
            except:
                plate_num = plate_num
        if plate_num != None:
            print("License Plate #: ", plate_num)
        #cv2.imshow("Character's Segmented", im2)
        #cv2.waitKey(0)
        return plate_num
    except:
        return ""
def yolobbox2bbox(x,y,w,h):
    x1, y1 = x-w/2, y-h/2
    x2, y2 = x+w/2, y+h/2
    return x1, y1, x2, y2

def get_data(distros_dict):
    json_to_variable = []
    # For every frame.
    count = 0
    for distro in distros_dict:
        filename = distro['filename']
        now_filename_arr = filename.split('/')
        now_filename = now_filename_arr[1]+'/'+now_filename_arr[2]
        img = cv2.imread(now_filename)
        # I have opened the image
        if len(distro['objects']) != 0:
            # For every detection.
            for obj in range(len(distro['objects'])):
                # Get values.
                x = distro['objects'][obj]["relative_coordinates"]["center_x"]
                y = distro['objects'][obj]["relative_coordinates"]["center_y"]
                width = distro['objects'][obj]["relative_coordinates"]["width"]
                height = distro['objects'][obj]["relative_coordinates"]["height"]
                confidence = distro['objects'][obj]["confidence"]
                # img = cv2.imread(img)

                im=Image.open(now_filename)
                w= int(im.size[0])
                h= int(im.size[1])
                try:
                    x1 , y1, x2 , y2 = pbx.convert_bbox((x,y,width,height), from_type="yolo", to_type="voc", image_size=(w,h))
                except:
                    continue
                plate_num = recognize_plate(img,[x1,y1,x2,y2])


                xmin = round(x - (width / 2))
                xmax = round(x + (width / 2))
                ymin = round(y - (height / 2))
                ymax = round(y + (height / 2))

                cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0),1)
                cv2.putText(img, plate_num,
                    (x1,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 0, 255) ,2,cv2.LINE_AA)
        now = filename.split('/')
        Image.fromarray(img).save('paddle/'+now[-1])

        # If you need to use json_to_variable here, move "json_to_variable = []" inside "for distro in distros_dict:"
    # Add your code here.
    # Or return json_to_variable to use it outside this function.

with open('train.json', 'r') as f:
    distros_dict_train = json.load(f)
    
get_data(distros_dict_train)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster wi

In [1]:
import json
import glob
import cv2
import pytesseract
from PIL import Image
import pybboxes as pbx
import re
import numpy as np
import imutils 
import easyocr
reader = easyocr.Reader(['en'])

def find(s, ch,ch2):
    arr = [i for i, ltr in enumerate(s) if ltr == ch]
    s2 = ""
    for idx in range(0,len(s)):
        if(idx in arr):
            s2 += ch2
        else:
            s2 += s[idx]
    # print(s2)
    return s2

def recognize_plate(img, coords):
    
    # For indian images
    try:
        xmin, ymin, xmax, ymax = coords
        pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
        box = img[int(ymin):int(ymax), int(xmin)-5:int(xmax)+5]
            # print(box)
            
            # grayscale region within bounding box

        gray = cv2.cvtColor(box, cv2.COLOR_RGB2GRAY)
        # resize image to three times as large as original for better readability
        gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
        # perform gaussian blur to smoothen image
        blur = cv2.GaussianBlur(gray, (5,5), 0)
        #cv2.imshow("Gray", gray)
        #cv2.waitKey(0)
        # threshold the image using Otsus method to preprocess for tesseract
        ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
        #cv2.imshow("Otsu Threshold", thresh)
        #cv2.waitKey(0)
        # create rectangular kernel for dilation
        rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
        dilation = cv2.dilate(thresh, rect_kern, iterations = 1)
        reader = easyocr.Reader(['en'])
        result = reader.readtext(box)
        text = result[0][1].upper()
        text = text.strip()
        if(len(text) == 10):
            text_temp = text[0] + text[1] + text[4] + text[5]
            # getting all the letters
            find(text_temp,'5','S')
            find(text_temp,'7','Z')
            find(text_temp,'1','I')
            find(text_temp,'8','B')
            find(text_temp,'2','Z')
            find(text_temp,'4','A')
            find(text_temp,'6','G')

            text_temp2 = text[2] + text[3] + text[6] + text[7] + text[8] + text[9]
            find(text_temp2,'Q','0')
            find(text_temp2,'D','0')
            find(text_temp2,'Z','7')
            find(text_temp2,'S','5')
            find(text_temp2,'J','1')
            find(text_temp2,'I','1')
            find(text_temp,'A','4')
            find(text_temp,'B','8')

            new_text = text_temp[0] + text_temp[1] + text_temp2[0] + text_temp2[1] + text_temp[2] + text_temp[3] + text_temp2[2]+ text_temp2[3] + text_temp2[4] + text_temp2[5]
            print(new_text+"New")
            return new_text
        else:
            print(text)
            return text
    except:
        print("hello")
        return ""
# 
    # separate coordinates from box
    xmin, ymin, xmax, ymax = coords
    # print(img)
    # print(coords)
    # get the subimage that makes up the bounded region and take an additional 5 pixels on each side
    try:
        box = img[int(ymin):int(ymax), int(xmin)-5:int(xmax)+5]
        # print(box)
        
        # grayscale region within bounding box
        gray = cv2.cvtColor(box, cv2.COLOR_RGB2GRAY)
        # resize image to three times as large as original for better readability
        gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
        # perform gaussian blur to smoothen image
        blur = cv2.GaussianBlur(gray, (5,5), 0)
        #cv2.imshow("Gray", gray)
        #cv2.waitKey(0)
        # threshold the image using Otsus method to preprocess for tesseract
        ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
        #cv2.imshow("Otsu Threshold", thresh)
        #cv2.waitKey(0)
        # create rectangular kernel for dilation
        rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
        # apply dilation to make regions more clear
        dilation = cv2.dilate(thresh, rect_kern, iterations = 1)
        #cv2.imshow("Dilation", dilation)
        #cv2.waitKey(0)
        # find contours of regions of interest within license plate
        try:
            contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        except:
            ret_img, contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # sort contours left-to-right
        sorted_contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])
        # create copy of gray image
        im2 = gray.copy()
        # create blank string to hold license plate number
        plate_num = ""
        # loop through contours and find individual letters and numbers in license plate
        for cnt in sorted_contours:
            # try:
                x,y,w,h = cv2.boundingRect(cnt)
                height, width = im2.shape
                # if height of box is not tall enough relative to total height then skip
                if height / float(h) > 6: continue

                ratio = h / float(w)
                # if height to width ratio is less than 1.5 skip
                if ratio < 1.5: continue

                # if width is not wide enough relative to total width then skip
                if width / float(w) > 15: continue

                area = h * w
                # if area is less than 100 pixels skip
                if area < 100: continue

                # draw the rectangle
                rect = cv2.rectangle(im2, (x,y), (x+w, y+h), (0,255,0),2)
                # grab character region of image
                roi = thresh[y-5:y+h+5, x-5:x+w+5]
                # perfrom bitwise not to flip image to black text on white background
                roi = cv2.bitwise_not(roi)
                # perform another blur on character region
                roi = cv2.medianBlur(roi, 5)
                text = pytesseract.image_to_string(roi, config='-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ --psm 8 --oem 3')
                # clean tesseract text by removing any unwanted blank spaces
                clean_text = re.sub('[\W_]+', '', text)
                plate_num += clean_text
            # except:
                plate_num = plate_num
        if plate_num != None:
            print("License Plate #: ", plate_num)
        #cv2.imshow("Character's Segmented", im2)
        #cv2.waitKey(0)
        return plate_num
    except:
        return ""
def yolobbox2bbox(x,y,w,h):
    x1, y1 = x-w/2, y-h/2
    x2, y2 = x+w/2, y+h/2
    return x1, y1, x2, y2

def get_data(distros_dict):
    json_to_variable = []
    # For every frame.
    count = 0
    for distro in distros_dict:
        filename = distro['filename']
        now_filename_arr = filename.split('/')
        now_filename = now_filename_arr[1]+'/'+now_filename_arr[2]
        img = cv2.imread(now_filename)
        # I have opened the image
        if len(distro['objects']) != 0:
            # For every detection.
            for obj in range(len(distro['objects'])):
                # Get values.
                x = distro['objects'][obj]["relative_coordinates"]["center_x"]
                y = distro['objects'][obj]["relative_coordinates"]["center_y"]
                width = distro['objects'][obj]["relative_coordinates"]["width"]
                height = distro['objects'][obj]["relative_coordinates"]["height"]
                confidence = distro['objects'][obj]["confidence"]
                # img = cv2.imread(img)

                im=Image.open(now_filename)
                w= int(im.size[0])
                h= int(im.size[1])
                try:
                    x1 , y1, x2 , y2 = pbx.convert_bbox((x,y,width,height), from_type="yolo", to_type="voc", image_size=(w,h))
                except:
                    continue
                plate_num = recognize_plate(img,[x1,y1,x2,y2])


                xmin = round(x - (width / 2))
                xmax = round(x + (width / 2))
                ymin = round(y - (height / 2))
                ymax = round(y + (height / 2))
                print(plate_num)
                cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0),1)
                cv2.putText(img, plate_num,
                    (x1,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255) ,2,cv2.LINE_AA)
        now = filename.split('/')
        Image.fromarray(img).save('research_predictions/'+now[-1])

        # If you need to use json_to_variable here, move "json_to_variable = []" inside "for distro in distros_dict:"
    # Add your code here.
    # Or return json_to_variable to use it outside this function.

with open('test.json', 'r') as f:
    distros_dict_train = json.load(f)
    
get_data(distros_dict_train)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


KLO1CA2555New
KLO1CA2555


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


PGOMN112
PGOMN112


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


...

P28 CF 037New
P28 CF 037


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


P28 CF 037New
P28 CF 037


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


FU4 AE'744New
FU4 AE'744


In [8]:
import numpy as np
import cv2

# Match contours to license plate or character template
def find_contours(dimensions, img) :

    # Find all contours in the image
    cntrs, _ = cv2.findContours(img.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Retrieve potential dimensions
    lower_width = dimensions[0]
    upper_width = dimensions[1]
    lower_height = dimensions[2]
    upper_height = dimensions[3]
    

    # Check largest 5 or  15 contours for license plate or character respectively
    cntrs = sorted(cntrs, key=cv2.contourArea, reverse=True)[:15]

    x_cntr_list = []
    target_contours = []
    img_res = []
    for cntr in cntrs :
        #detects contour in binary image and returns the coordinates of rectangle enclosing it
        intX, intY, intWidth, intHeight = cv2.boundingRect(cntr)
        
        #checking the dimensions of the contour to filter out the characters by contour's size
        if intWidth > lower_width and intWidth < upper_width and intHeight > lower_height and intHeight < upper_height :
            x_cntr_list.append(intX) #stores the x coordinate of the character's contour, to used later for indexing the contours

            char_copy = np.zeros((44,24))
            #extracting each character using the enclosing rectangle's coordinates.
            char = img[intY:intY+intHeight, intX:intX+intWidth]
            char = cv2.resize(char, (20, 40))

            # Make result formatted for classification: invert colors
            char = cv2.subtract(255, char)

            # Resize the image to 24x44 with black border
            char_copy[2:42, 2:22] = char
            char_copy[0:2, :] = 0
            char_copy[:, 0:2] = 0
            char_copy[42:44, :] = 0
            char_copy[:, 22:24] = 0

            img_res.append(char_copy) #List that stores the character's binary image (unsorted)

    #Return characters on ascending order with respect to the x-coordinate (most-left character first)
    
    #arbitrary function that stores sorted list of character indeces
    indices = sorted(range(len(x_cntr_list)), key=lambda k: x_cntr_list[k])
    img_res_copy = []
    for idx in indices:
        img_res_copy.append(img_res[idx])# stores character images according to their index
    img_res = np.array(img_res_copy)

    return img_res


In [5]:

# Find characters in the resulting images
import cv2
def segment_characters(image) :

    # Preprocess cropped license plate image
    img = cv2.resize(image, (333, 75))
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, img_binary = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img_erode = cv2.erode(img_binary, (3,3))
    img_dilate = cv2.dilate(img_erode, (3,3))

    LP_WIDTH = img_dilate.shape[0]
    LP_HEIGHT = img_dilate.shape[1]

    # Make borders white
    img_dilate[0:3,:] = 255
    img_dilate[:,0:3] = 255
    img_dilate[72:75,:] = 255
    img_dilate[:,330:333] = 255

    # Estimations of character contours sizes of cropped license plates
    dimensions = [LP_WIDTH/6, LP_WIDTH/2, LP_HEIGHT/10, 2*LP_HEIGHT/3]

    # Get contours within cropped license plate
    char_list = find_contours(dimensions, img_dilate)

    return char_list


In [6]:
def fix_dimension(img): 
  new_img = np.zeros((28,28,3))
  for i in range(3):
    new_img[:,:,i] = img
  return new_img

In [10]:
import pickle
with open('file1.pickle', 'rb') as f:
    model = pickle.load(f) 

In [13]:
import json
import glob
import cv2
import pytesseract
from PIL import Image
import pybboxes as pbx
import re
import numpy as np
import imutils 
import easyocr

def find(s, ch,ch2):
    arr = [i for i, ltr in enumerate(s) if ltr == ch]
    s2 = ""
    for idx in range(0,len(s)):
        if(idx in arr):
            s2 += ch2
        else:
            s2 += s[idx]
    # print(s2)
    return s2

def recognize_plate(img, coords):
    
    # For indian images
    # try:
    #     xmin, ymin, xmax, ymax = coords
    #     pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
    #     box = img[int(ymin):int(ymax), int(xmin)-5:int(xmax)+5]
    #         # print(box)
            
    #         # grayscale region within bounding box
    #     gray = cv2.cvtColor(box, cv2.COLOR_RGB2GRAY)
    #     # resize image to three times as large as original for better readability
    #     gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
    #     # perform gaussian blur to smoothen image
    #     blur = cv2.GaussianBlur(gray, (5,5), 0)
    #     #cv2.imshow("Gray", gray)
    #     #cv2.waitKey(0)
    #     # threshold the image using Otsus method to preprocess for tesseract
    #     ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
    #     #cv2.imshow("Otsu Threshold", thresh)
    #     #cv2.waitKey(0)
    #     # create rectangular kernel for dilation
    #     rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    #     dilation = cv2.dilate(thresh, rect_kern, iterations = 1)
    #     text = pytesseract.image_to_string(dilation, config='-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ --psm 8 --oem 3')
    #     text = text.capitalize()
    #     if(len(text) == 10):
    #         text_temp = text[0] + text[1] + text[4] + text[5]
    #         # getting all the letters
    #         find(text_temp,'5','S')
    #         find(text_temp,'7','Z')
    #         find(text_temp,'1','I')
    #         find(text_temp,'8','B')
    #         find(text_temp,'2','Z')
    #         find(text_temp,'4','A')
    #         find(text_temp,'6','G')

    #         text_temp2 = text[2] + text[3] + text[6] + text[7] + text[8] + text[9]
    #         find(text_temp2,'Q','0')
    #         find(text_temp2,'D','0')
    #         find(text_temp2,'Z','7')
    #         find(text_temp2,'S','5')
    #         find(text_temp2,'J','1')
    #         find(text_temp2,'I','1')
    #         find(text_temp,'A','4')
    #         find(text_temp,'B','8')

    #         new_text = text_temp[0] + text_temp[1] + text_temp2[0] + text_temp2[1] + text_temp[2] + text_temp[3] + text_temp2[2]+ text_temp2[3] + text_temp2[4] + text_temp2[5]
    xmin, ymin, xmax, ymax = coords

    # try:
    box = img[int(ymin):int(ymax), int(xmin)-5:int(xmax)+5]
    char = segment_characters(box)
    dic = {}
    characters = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    for i,c in enumerate(characters):
        dic[i] = c

    output = []
    for i,ch in enumerate(char): #iterating over the characters
        img_ = cv2.resize(ch, (28,28))
        img = fix_dimension(img_)
        img = img.reshape(1,28,28,3) #preparing image for the model
        y_ = model.predict(img)[0] #predicting the class
        idx = -1
        cnt = 0
        for ele in y_:
            if(ele == 1):
                idx= cnt
            cnt = cnt+1
        if(idx == -1):
            continue
        output.append(dic[idx]) #storing the result in a list
        
    plate_number = ''.join(output)
    return plate_number
    # except:
    #     return ""
def yolobbox2bbox(x,y,w,h):
    x1, y1 = x-w/2, y-h/2
    x2, y2 = x+w/2, y+h/2
    return x1, y1, x2, y2

def get_data(distros_dict):
    json_to_variable = []
    # For every frame.
    count = 0
    for distro in distros_dict:
        filename = distro['filename']
        now_filename_arr = filename.split('/')
        now_filename = now_filename_arr[1]+'/'+now_filename_arr[2]
        img = cv2.imread(now_filename)
        # I have opened the image
        if len(distro['objects']) != 0:
            # For every detection.
            for obj in range(len(distro['objects'])):
                # Get values.
                x = distro['objects'][obj]["relative_coordinates"]["center_x"]
                y = distro['objects'][obj]["relative_coordinates"]["center_y"]
                width = distro['objects'][obj]["relative_coordinates"]["width"]
                height = distro['objects'][obj]["relative_coordinates"]["height"]
                confidence = distro['objects'][obj]["confidence"]
                # img = cv2.imread(img)

                im=Image.open(now_filename)
                w= int(im.size[0])
                h= int(im.size[1])
                try:
                    x1 , y1, x2 , y2 = pbx.convert_bbox((x,y,width,height), from_type="yolo", to_type="voc", image_size=(w,h))
                except:
                    continue
                plate_num = recognize_plate(img,[x1,y1,x2,y2])


                xmin = round(x - (width / 2))
                xmax = round(x + (width / 2))
                ymin = round(y - (height / 2))
                ymax = round(y + (height / 2))
                print(plate_num)
                cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0),1)
                cv2.putText(img, plate_num,
                    (x1,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255) ,2,cv2.LINE_AA)
        now = filename.split('/')
        Image.fromarray(img).save('new_cnn/'+now[-1])

        # If you need to use json_to_variable here, move "json_to_variable = []" inside "for distro in distros_dict:"
    # Add your code here.
    # Or return json_to_variable to use it outside this function.

with open('test.json', 'r') as f:
    distros_dict_train = json.load(f)
    
get_data(distros_dict_train)

1/1 [==============================] - 0s 41ms/step
KE01EA2555
1/1 [==============================] - 0s 45ms/step
PGMN112


1/1 [==============================] - 0s 22ms/step
HG6861T3
1/1 [==============================] - 0s 24ms/step
0

1/1 [==============================] - 0s 23ms/step
HL60LX
1/1 [==============================] - 0s 23ms/step
4H2C






1/1 [==============================] - 0s 21ms/step
HH01AY8866
1/1 [==============================] - 0s 28ms/step
CZ7K00Q
1/1 [==============================] - 0s 23ms/step
90R56K
1/1 [==============================] - 0s 30ms/step
A
1/1 [==============================] - 0s 19ms/step
RH28EE7598
1/1 [==============================] - 0s 24ms/step
A1
1/1 [==============================] - 0s 22ms/step
PE

1/1 [==============================] - 0s 25ms/step
Y00
1/1 [==============================] - 0s 22ms/step
773W7KK



1/1 [==============================] - 0s 20ms/step
4HH12867237
1/1 [==============================] - 0s 21

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
